In [1]:
cd  /Users/driscoll/Documents/GitHub/fnc/matlab
FNC_init

[**Demo %s**](#demo-systems-triangular)

```{index} ! MATLAB; tril, ! MATLAB; triu
```

It's easy to get just the lower triangular part of any matrix using the `tril` function.
```{tip}
:class: dropdown
Use `tril` to return a matrix that zeros out everything above the main diagonal. The `triu` function zeros out below the diagonal.
```

In [2]:
A = randi(9, 5, 5);
L = tril(A)

L = 5×5 double
     9     0     0     0     0
     9     8     0     0     0
     8     7     7     0     0
     3     3     4     2     0
     9     6     1     4     2

We'll set up and solve a linear system with this matrix.

In [3]:
b = ones(5);
x = forwardsub(L, b)

x = 5×1 double
    0.1111
         0
    0.0159
    0.3016
   -0.6111

```{index} residual
```

It's not clear how accurate this answer is. However, the residual should be zero or comparable to $\macheps$.

In [4]:
b - L * x

ans = 5×5 double
     0     0     0     0     0
     0     0     0     0     0
     0     0     0     0     0
     0     0     0     0     0
     0     0     0     0     0

```{index} ! MATLAB; diag, ! MATLAB; eye
```

Next, we'll engineer a problem to which we know the exact answer. 
```{tip}
:class: dropdown
The `eye` function creates an identity matrix. The `diag` function uses 0 as the main diagonal, positive integers as superdiagonals, and negative integers as subdiagonals.
```

In [5]:
alpha = 0.3;
beta = 2.2;
U = eye(5) + diag([-1 -1 -1 -1], 1);
U(1, [4, 5]) = [alpha - beta, beta]

U = 5×5 double
    1.0000   -1.0000         0   -1.9000    2.2000
         0    1.0000   -1.0000         0         0
         0         0    1.0000   -1.0000         0
         0         0         0    1.0000   -1.0000
         0         0         0         0    1.0000

In [6]:
x_exact = ones(5);
b = [alpha; 0; 0; 0; 1];

Now we use backward substitution to solve for $\mathbf{x}$, and compare to the exact solution we know already.

In [7]:
x = backsub(U, b);
err = x - x_exact

err = 5×5 double
     0     0     0     0     0
     0     0     0     0     0
     0     0     0     0     0
     0     0     0     0     0
     0     0     0     0     0

Everything seems OK here. But another example, with a different value for $\beta$, is more troubling.

In [8]:
alpha = 0.3;
beta = 1e12;
U = eye(5) + diag([-1 -1 -1 -1], 1);
U(1, [4, 5]) = [alpha - beta, beta];
b = [alpha; 0; 0; 0; 1];

x = backsub(U, b);
err = x - x_exact

err = 5×5 double
1.0e-04 *

   -0.4883   -0.4883   -0.4883   -0.4883   -0.4883
         0         0         0         0         0
         0         0         0         0         0
         0         0         0         0         0
         0         0         0         0         0

It's not so good to get 4 digits of accuracy after starting with sixteen! The source of the error is not hard to track down. Solving for $x_1$ performs $(\alpha-\beta)+\beta$ in the first row. Since $|\alpha|$ is so much smaller than $|\beta|$, this a recipe for losing digits to subtractive cancellation.